# 파일 불러오기

데이터 원본 파일을 불러와서 SENT에 해당하는 문장만 리스트로 변환합니다

In [ ]:
import pandas as pd

dataset = pd.read_excel('/content/data2.xlsx')
dataset

In [ ]:
sent = dataset['SENT']
sent_list = sent.values.tolist()
len(sent_list)

# 전처리

콤마 앞, 뒤로 ## 태그 형식을 통일하고 이외에 불필요한 개행문자, 중복 쉼표 등을 1개의 쉼표로 바꿉니다

더 필요한 전처리를 추가해주시면 됩니다

In [4]:
import re

prep1 = []

def prep_comma(data):
  for i in data:
    i = re.sub('##，##', '，##', i)
    i = re.sub('##，', '，##', i)
    i = re.sub('\n', '，', i)
    i = re.sub('，，', '，', i)
    prep1.append(i)

prep_comma(sent_list)

# 태그 기본 형식에 안 맞는 경우

예시 문장 : ##这时/T##她/A##索性把自己的思想简单化了一下

첫번째 chunk : ##这时/T

두번째 chunk : ##她/A

나머지 문장 : ##索性把自己的思想简单化了一下

따라서, ## 태그와 의미역 태그 /대문자 의 차이는 1개가 되어야 합니다

이 논리에 맞지 않는 경우와 의미역 태그가 아예 없거나 ##이 없는 경우 모두 wrong_list에 저장합니다

In [ ]:
wrong_list = []
wrong_idx = []

for i in prep1:
  if i.count('##') - i.count('/') != 1 or i.count('/') == 0 or i.count('##') == 0 :
    if i.count('[a-zA-Z]') - i.count('/') != 0:
      wrong_list.append(i)
      wrong_idx.append(prep1.index(i))

import pandas as pd

df = pd.DataFrame({"Index" : wrong_idx, "Sentence" : wrong_list})
df

In [18]:
path = '/content/'
df.to_csv(path + "data2_검토필요.csv", encoding="utf-8-sig", index=False)

# 논항 2개 swap

First    :  ##小时候/T

Second   :  ##他/A

Original :  ##小时候/T，##他/A##也曾跟父母到过上海的梯梯斯、百乐门和法国夜总会，现在应该像是旧地重游

Changed  :  ##他/A，##小时候/T##也曾跟父母到过上海的梯梯斯、百乐门和法国夜总会，现在应该像是旧地重游

위와 같이, 원문장에서 2개의 의미역 덩어리를 추출해서 서로 swap 하는 코드입니다.

In [ ]:
ck1 = []
args2 = []
args2_idx = []
original = []
first_elem = []
second_elem = []
args2_swap = []

for i in prep1:
  if i.count('/') == 2:
    if i.count('##') == 3:
      if len(re.findall('[a-zA-Z]', i)) == 2:
        #print(i)
        args2.append(i)
        args2_idx.append(prep1.index(i))
      else:
        ck1.append(i)

for i in args2:
  if '/' in i:
    sharp1 = i.index('##') 
    first = re.search('[a-zA-Z]', i)
    mean1 = i.index(first.group())
    remain = str(i[mean1 + 1:])
    elem1 = i[sharp1:mean1 + 1]      
    print("\nFirst    : ", elem1)
    
  if '/' in remain:
    sharp2 = remain.index('##')
    second = re.search('[a-zA-Z]', remain)
    mean2 = remain.index(second.group())
    remain2 = str(remain[mean2 + 1:])
    elem2 = remain[sharp2:mean2 + 1]
    data1 = i.replace(elem1, '@').replace(elem2, elem1).replace('@', elem2) # ba       
    print("Second   : ", elem2)
    print("Original : ", i)
    print("Changed  : ", data1) 
    original.append(i)
    first_elem.append(elem1)
    second_elem.append(elem2)
    args2_swap.append(data1)

# 논항 2개 swap 결과 저장

In [ ]:
import pandas as pd
df = pd.DataFrame({"Index" : args2_idx, "Original" : args2, "Swapped" : args2_swap})
df

In [ ]:
path = '/content/'
df.to_csv(path + "data2_mean2_tag2.csv", encoding="utf-8-sig", index=False)

# 논항 3개 swap

First    :  ##这时/T

Second   :  ##柳清/E

Third   :  ##心里/E

Original :  ##这时/T##柳清/E##心里/E##特别沉重

Changed  : 
 ##这时/T##心里/E##柳清/E##特别沉重 
 ##柳清/E##这时/T##心里/E##特别沉重 
 ##柳清/E##心里/E##这时/T##特别沉重 
 ##心里/E##这时/T##柳清/E##特别沉重 
 ##心里/E##柳清/E##这时/T##特别沉重

 위와 같이, 원문장에서 3개의 의미역 덩어리를 추출해 swap 하는 코드입니다.

In [ ]:
ck2 = []
args3 = []
args3_idx = []
args3_original = []
args3_first = []
args3_second = []
args3_third = []
args3_swap = []
args3_wrong = []

for i in prep1:
  if i.count('/') == 3:
    if i.count('##') == 4:
      if len(re.findall('[a-zA-Z]', i)) == 3:
        #print(i)
        args3.append(i)
        args3_idx.append(prep1.index(i))
      else:
        ck2.append(i)

for i in args3:
  if '/' in i:
    sharp1 = i.index('##') 
    first = re.search('[a-zA-Z]', i)
    mean1 = i.index(first.group())
    remain = str(i[mean1 + 1:])
    elem1 = i[sharp1:mean1 + 1]      
    print("\nFirst    : ", elem1)
    
  if '/' in remain:
    sharp2 = remain.index('##')
    second = re.search('[a-zA-Z]', remain)
    mean2 = remain.index(second.group())
    remain2 = str(remain[mean2 + 1:])
    elem2 = remain[sharp2:mean2 + 1]
    data1 = i.replace(elem1, '@').replace(elem2, elem1).replace('@', elem2) # ba       
    print("Second   : ", elem2)
    
  if '/' in remain2:
    sharp3_num = remain2.index('##')
    third = re.search('[a-zA-Z]', remain2)
    mean3_num = remain2.index(third.group())
    remain3 = str(remain2[mean3_num + 1:])
    elem3 = remain2[sharp3_num:mean3_num + 1]
    data2_1 = i.replace(elem2, '@').replace(elem3, elem2).replace('@', elem3) #acb
    data2_2 = i.replace(elem1, '@').replace(elem2, elem1).replace('@', elem2) #bac
    data2_3 = i.replace(elem2, '@').replace(elem1, elem2).replace(elem3, elem1).replace('@', elem3) #bca
    data2_4 = i.replace(elem2, '@').replace(elem3, elem2).replace(elem1, elem3).replace('@', elem1) #cab
    data2_5 = i.replace(elem1, '@').replace(elem3, elem1).replace('@', elem3) #cba        
    print("Third   : ", elem3)
    print("Original : ", i)
    print("Changed  : \n", data2_1, "\n" ,data2_2, "\n", data2_3, "\n", data2_4, "\n", data2_5)
    args3_original.append(i)
    args3_first.append(elem1)
    args3_second.append(elem2)
    args3_third.append(elem3)
    args3_swap.append(data2_1 + "\n" + data2_2 + "\n" + data2_3 + "\n" + data2_4 + "\n" + data2_5) 

# 논항 3개 swap 결과 저장

In [ ]:
import pandas as pd
df = pd.DataFrame({"Index" : args3_idx, "Original" : args3, "Swapped" : args3_swap})
df

In [ ]:
path = '/content/'
df.to_csv(path + "data2_mean3_tag3.csv", encoding="utf-8-sig", index=False)